<a href="https://colab.research.google.com/github/lakshitgosain/Tensorflow-ZTM/blob/main/TF_ZTM_07_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP Fundamentals in TF

Sequence-to-sequence
* One to one- Image Captioning
* Many to one- Sentiment analysis
* Many to one- time series forecasting
* many- to-many- Machine translation


In [90]:
!nvidia-smi -L

/bin/bash: nvidia-smi: command not found


In [91]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-06-16 03:12:50--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-06-16 03:12:50 (66.8 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



In [92]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [93]:
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

--2023-06-16 03:12:50--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.107.128, 74.125.196.128, 74.125.134.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.107.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.1’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2023-06-16 03:12:51 (102 MB/s) - ‘nlp_getting_started.zip.1’ saved [607343/607343]



In [94]:
unzip_data("nlp_getting_started.zip")

## Visualize the dataset

To visualize the sample data, we need to read the in. On way is to use python.

Another way to do this is use pandas



In [95]:
import pandas as pd
train_df=pd.read_csv("train.csv")
test_df=pd.read_csv("test.csv")

In [96]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [97]:
train_data_shuffled=train_df.sample(frac=1, random_state=42)

In [98]:
train_data_shuffled

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0
...,...,...,...,...,...
5226,7470,obliteration,Merica!,@Eganator2000 There aren't many Obliteration s...,0
5390,7691,panic,NaN,just had a panic attack bc I don't have enough...,0
860,1242,blood,NaN,Omron HEM-712C Automatic Blood Pressure Monito...,0
7603,10862,NaN,NaN,Officials say a quarantine is in place at an A...,1


In [99]:
#Test dataframe looks like:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [100]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [101]:
#How many total number of samples
len(train_df), len(test_df)

(7613, 3263)

In [102]:
#Let's visualize some random training samples
import random
random_index = random.randint(0,len(train_df)-5)
for row in train_data_shuffled[['text','target']][random_index:random_index+5].itertuples():
  _,text, target=row
  print(f"Target: {target}", "(real disaster)" if target >0 else "(not real disaster)")
  print(f"text:\n{text}\n")
  print("---\n")

Target: 1 (real disaster)
text:
Wreckage 'Conclusively Confirmed' as From MH370: Malaysia PM: Investigators and the families of those who were... http://t.co/MSsq0sVnBM

---

Target: 1 (real disaster)
text:
Russian 'food crematoria' provoke outrage amid crisis famine memories - Yahoo News http://t.co/6siiRlnV6z

---

Target: 1 (real disaster)
text:
The ol' meltdown victory for the Mets.

---

Target: 1 (real disaster)
text:
More Natural Disaster Research Urgent http://t.co/5Cm0LfZhxn via #JakartaPost

---

Target: 0 (not real disaster)
text:
I have been bleeding into this typewriter all day but so far all I've written is a bunch of gunk.

---



### Split data into training and validation sets



In [103]:
from sklearn.model_selection import train_test_split

In [104]:
train_sentences, val_sentences, train_labels, val_labels=train_test_split(train_data_shuffled['text'].to_numpy(),
                                                                          train_data_shuffled['target'].to_numpy(),
                                                                          test_size=0.1,
                                                                          random_state=42)

In [105]:
#Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

## Converting words into numbers

First thing while building a model is to convert your text into numbers


Tokenization VS Embedding
Tkenization - word level and character level. Direct apping of a token.

Embedding - every word gets turned into a vector and we can define size of the vector. Embeddings can learn as our model trains



Text vectorization(tokenization)

In [106]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [107]:
#Use the default text vectorization parameters
text_vectoizer= TextVectorization(max_tokens=None, #Defines how many words in our vocab
                               standardize= "lower_and_strip_punctuation",
                               split="whitespace",
                               ngrams=None,
                               output_mode='int',
                               output_sequence_length=None) #how long do you want your sequences to be
                               #pad_to_max_tokens=True)

In [108]:
train_sentences[0].split()

['@mogacola', '@zamtriossu', 'i', 'screamed', 'after', 'hitting', 'tweet']

In [109]:
#Find the average number of tokens(words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [110]:
max_vocab_length= 10000 #max no of words to have in our vocab
max_length=15 #Max length our sequences will be (e.g. how many words from a Tweet does a model see)

text_vectorizer=TextVectorization(max_tokens=max_vocab_length,
                                     output_mode="int",
                                     output_sequence_length=max_length)

In [111]:
#Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)


In [112]:
train_sentences[:10]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk',
       '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
       'destroy the free fandom honestly',
       'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
       '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
       'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
      dtype=object)

In [113]:
#Create a sample sentence and tokenize it
sample_sentence="There is a floow in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 74,   9,   3,   1,   4,  13, 698,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [114]:
random_sentence=random.choice(train_sentences)
print(random_sentence)
vectorized_random_sentence=text_vectorizer([random_sentence])
print(f"Random Sentence: {random_sentence}\nVectoized Random Sentence: {vectorized_random_sentence}")

FedEx no longer will ship potential bioterror pathogens - FedEx Corp. (NYSE: FDX) will no longer deliver packages ... http://t.co/2kdq56xTWs
Random Sentence: FedEx no longer will ship potential bioterror pathogens - FedEx Corp. (NYSE: FDX) will no longer deliver packages ... http://t.co/2kdq56xTWs
Vectoized Random Sentence: [[ 577   40  600   38  810 1615  609 1801  577 5912    1    1   38   40
   600]]


In [115]:
words_in_vocab=text_vectorizer.get_vocabulary()
top_5_words=words_in_vocab[:5]
botton_5_words=words_in_vocab[-5:]

In [116]:
words_in_vocab,top_5_words,botton_5_words

(['',
  '[UNK]',
  'the',
  'a',
  'in',
  'to',
  'of',
  'and',
  'i',
  'is',
  'for',
  'on',
  'you',
  'my',
  'with',
  'it',
  'that',
  'at',
  'by',
  'this',
  'from',
  'be',
  'are',
  'was',
  'have',
  'like',
  'as',
  'up',
  'so',
  'just',
  'but',
  'me',
  'im',
  'your',
  'not',
  'amp',
  'out',
  'its',
  'will',
  'an',
  'no',
  'has',
  'fire',
  'after',
  'all',
  'when',
  'we',
  'if',
  'now',
  'via',
  'new',
  'more',
  'get',
  'or',
  'about',
  'what',
  'he',
  'people',
  'news',
  'been',
  'over',
  'one',
  'how',
  'dont',
  'they',
  'who',
  'into',
  'were',
  'do',
  'us',
  '2',
  'can',
  'video',
  'emergency',
  'there',
  'disaster',
  'than',
  'police',
  'would',
  'his',
  'still',
  'her',
  'some',
  'body',
  'storm',
  'crash',
  'burning',
  'suicide',
  'back',
  'man',
  'california',
  'why',
  'time',
  'them',
  'had',
  'buildings',
  'rt',
  'first',
  'cant',
  'see',
  'got',
  'day',
  'off',
  'our',
  'going',
 

### Creating an embedding using an embedding layer

To make our embedding, we're going to use the TF's embedding layer.

The parameters we care the most about our embedding layers are:
* input_dim= sie of our vocab
* output_dim= size of the output embedding vector. A value of a 100 will mean that each token will be represented as a vector of 100 long length
* input_length= length of sequences being passed to the embedding layer
*

In [117]:
from tensorflow.keras import layers

In [118]:
embedding=layers.Embedding(input_dim=max_vocab_length,
                           output_dim=128,
                           input_length=max_length)

In [119]:
embedding

In [120]:
#Get a random sentence from the training set
random_sentence=random.choice(train_sentences)
print(f"Original Sentence:{random_sentence}\
      \nEmbedded Sentence")

sample_embedding=embedding(text_vectorizer([random_sentence]))
sample_embedding

Original Sentence:Ancient Mayan Tablet Found in Jungle Temple http://t.co/qp6q8RS8ON      
Embedded Sentence


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.03305274,  0.03334713, -0.0422186 , ..., -0.0002015 ,
         -0.02461776, -0.03663777],
        [-0.03749834,  0.04993585,  0.0281971 , ..., -0.01925508,
          0.00739387, -0.04485953],
        [ 0.02407706, -0.0220372 , -0.00616293, ..., -0.00148871,
          0.00904542,  0.00793977],
        ...,
        [ 0.01678229, -0.04126339,  0.04923167, ..., -0.01411662,
         -0.02210027, -0.00361538],
        [ 0.01678229, -0.04126339,  0.04923167, ..., -0.01411662,
         -0.02210027, -0.00361538],
        [ 0.01678229, -0.04126339,  0.04923167, ..., -0.01411662,
         -0.02210027, -0.00361538]]], dtype=float32)>

In [121]:
#Check out a single token's embedding
sample_embedding[0][0], sample_embedding[0][0].shape, random_sentence[0]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.03305274,  0.03334713, -0.0422186 , -0.01651434, -0.01004869,
         0.00807239,  0.02143629,  0.04050611, -0.0488528 ,  0.04384295,
        -0.04994569, -0.02975096, -0.04748983,  0.03848847,  0.014861  ,
        -0.03664464,  0.02235797, -0.01168202, -0.00961262, -0.02008259,
         0.04554153,  0.04856726,  0.000729  ,  0.015517  ,  0.02172233,
         0.01956419, -0.00673329, -0.01454289, -0.02853456, -0.01421569,
        -0.03547049, -0.04265514,  0.04614327, -0.02024713,  0.0492197 ,
         0.01474769, -0.03769908, -0.04338888,  0.00069172,  0.03630065,
         0.01816435,  0.01300832, -0.01224078,  0.04164369, -0.03761123,
        -0.03906231, -0.02251348,  0.01014551, -0.01259303,  0.01133149,
         0.02600336,  0.02457358, -0.03501425,  0.04379206,  0.04179703,
        -0.01283264,  0.02588114, -0.03014401, -0.0332365 , -0.00388657,
         0.03525729,  0.01430574, -0.03992382,  0.01723776,  0.0177853 ,
  

##Modelling a text dataset(running a series of experiments)

* Model 0: Naive Bayes with Tokenization
* Model 1: Feed-forward Neural Network(Dense Model)
* Model 2: LSTM Model(RNN)
* Model 3: GRU Model
* Moel 4: Bidirectional LSTM
* Model 5 : 1D Convolutional Layer
* model 6: Tensorflow hub pretrained feature extraction (using transfer learning)
* Model 7 : Same as model 6 with 10% of data

How are we going to approach all of these:
Use the standard steps in modelling with tensorflow

* Create->build->fit->Evaluate



### Model 0

To create a baseline, we'll use sklearn's Naive Bayes using Tf-IDF to convert our words to numbers

It is a good practice to use non-DL Algos as a baseline because of their speed and then later use DL to see if you can improve upon them

In [122]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [123]:
model_0=Pipeline([
    ("tfidf",TfidfVectorizer()),
    ("clf",MultinomialNB())
])

model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [124]:
#Evaluate the baseline Model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"our baseline model achieves an accuracy of : {baseline_score*100:.2f}%")

our baseline model achieves an accuracy of : 79.27%


In [125]:
train_df.target.value_counts

<bound method IndexOpsMixin.value_counts of 0       1
1       1
2       1
3       1
4       1
       ..
7608    1
7609    1
7610    1
7611    1
7612    1
Name: target, Length: 7613, dtype: int64>

In [126]:
#Make predictions
baseline_preds= model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Create an evaluation function for our model experiments

We could evaluate all of our model's predictions with different metrics every time.
Calculate:
* Accuracy
* Precision
* Recall
* F1 score


In [127]:
#Function to evaluate the above metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculated model accuracy, precision, recall and f1 score of a binary classification model
  """
  #Calculate model accuracy
  model_accuracy=accuracy_score(y_true, y_pred) * 100
  #Calculate model precision, recall and f1 score using the"weighted average"
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred,average="weighted" )
  model_results={'accuracy':model_accuracy,
                 'precision': model_precision,
                 'recall': model_recall,
                 'f1-score':model_f1}
  return model_results

In [128]:
#Get baseline results
baseline_results=calculate_results(y_true=val_labels,
                                   y_pred=baseline_preds)

In [129]:
baseline_results['accuracy']

79.26509186351706

In [130]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1-score': 0.7862189758049549}

## Model 1: A simple Dense Model

In [131]:
#Create a tensorflow callback
from helper_functions import create_tensorboard_callback

#Create a directory to save tensorboard logs
SAVE_DIR="model_logs"


In [132]:
#Build model with functional API
from tensorflow.keras import layers

inputs=layers.Input(shape=(1,), dtype=tf.string) #Inputs are 1 dimensional
x= text_vectorizer(inputs) #Turn the input text into numbers
x= embedding(x) #Create an embedding of vctorized embeddings
x=layers.GlobalAveragePooling1D()(x)
outputs=layers.Dense(1, activation='sigmoid')(x)

model_1=tf.keras.Model(inputs, outputs, name="Model_1_dense")

In [133]:
model_1.summary()

Model: "Model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_3 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d_1   (None, 128)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_16 (Dense)            (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [134]:
#Compile model
model_1.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [135]:
model_1_history=model_1.fit(x=train_sentences,
                            y=train_labels,
                            epochs=5,
                            validation_data=(val_sentences,val_labels),
                            callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                   experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20230616-031253
Epoch 1/5
215/215 [==============================] - 8s 31ms/step - loss: 0.6107 - accuracy: 0.6949 - val_loss: 0.5365 - val_accuracy: 0.7598
Epoch 2/5
215/215 [==============================] - 9s 42ms/step - loss: 0.4408 - accuracy: 0.8203 - val_loss: 0.4684 - val_accuracy: 0.7822
Epoch 3/5
215/215 [==============================] - 8s 36ms/step - loss: 0.3470 - accuracy: 0.8619 - val_loss: 0.4617 - val_accuracy: 0.7940
Epoch 4/5
215/215 [==============================] - 10s 47ms/step - loss: 0.2837 - accuracy: 0.8917 - val_loss: 0.4646 - val_accuracy: 0.7861
Epoch 5/5
215/215 [==============================] - 8s 39ms/step - loss: 0.2377 - accuracy: 0.9115 - val_loss: 0.4793 - val_accuracy: 0.7822


In [136]:
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 4ms/step - loss: 0.4793 - accuracy: 0.7822


[0.4793274998664856, 0.7821522355079651]

In [137]:
model_1_pred_probs=model_1.predict(val_sentences)

24/24 [==============================] - 1s 4ms/step


In [138]:
model_1_pred_probs.shape

(762, 1)

In [139]:
#calculate_results(val_labels, model_1_pred_probs)

ValueError: ignored

In [145]:
val_labels

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0,

In [146]:
#Convert Model_prediction Probabs into label formats
model_1_preds=tf.squeeze(tf.round(model_1_pred_probs))

In [147]:
model_1_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0

In [148]:
model_1_results=calculate_results(y_true=val_labels,
                                  y_pred=model_1_preds)

In [149]:
model_1_results

{'accuracy': 78.21522309711287,
 'precision': 0.7864332425219001,
 'recall': 0.7821522309711286,
 'f1-score': 0.7792361147360404}

In [150]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1-score': 0.7862189758049549}

In [151]:
import numpy as np
np.array(list(model_1_results.values()))> np.array(list(baseline_results.values()))

array([False, False, False, False])

## Visualize the learned embeddings


In [ ]:
#Get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()

In [ ]:
len(words_in_vocab), words_in_vocab[:10]

In [ ]:
# Model 1 summary
model_1.summary()

In [ ]:
#Get the weight matrix of the embedding layers
#These are numerical representation of our numerical datawhich are trained for 5 epochs

embed_weights=model_1.get_layer("embedding").get_weights()[0]
embed_weights

In [ ]:
print(embed_weights.shape)

Now we've got the embedding matrix our model has learned to represent tokens, let's see how we can visualize it

To do so, tensorflow has a tool called Projector

In [ ]:
# # Code below is adapted from: https://www.tensorflow.org/tutorials/text/word_embeddings#retrieve_the_trained_word_embeddings_and_save_them_to_disk
import io

# Create output writers
out_v = io.open("embedding_vectors.tsv", "w", encoding="utf-8")
out_m = io.open("embedding_metadata.tsv", "w", encoding="utf-8")

# Write embedding vectors and words to file
for num, word in enumerate(words_in_vocab):
  if num == 0:
     continue # skip padding token
  vec = embed_weights[num]
  out_m.write(word + "\n") # write words to file
  out_v.write("\t".join([str(x) for x in vec]) + "\n") # write corresponding word vector to file
out_v.close()
out_m.close()

In [ ]:
# Download files locally to upload to Embedding Projector
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download("embedding_vectors.tsv")
  files.download("embedding_metadata.tsv")

## Recurrent Neural Networks(RNNs)

the premise of the Recurrent Neural Networs is to use the representation of the previous input to aid the representation of the later input

RNNs are used for sequence data (this example is a sequence of text)

 📖 Resources:

MIT Deep Learning Lecture on Recurrent Neural Networks - explains the background of recurrent neural networks and introduces LSTMs.
The Unreasonable Effectiveness of Recurrent Neural Networks by Andrej Karpathy - demonstrates the power of RNN's with examples generating various sequences.
Understanding LSTMs by Chris Olah - an in-depth (and technical) look at the mechanics of the LSTM cell, possibly the most popular RNN building block.

### Model 2: LSTM

LSTM= long short term memory

Our structure of an RNN Looks like this

Input(text)-> Tokenize -> Embedding -> Layers(RNNs/Dense)->output(label probability)

In [ ]:
#Create an LSTM Model
from tensorflow.keras import layers
inputs= layers.Input(shape=(1,), dtype=tf.string)
x=text_vectorizer(inputs)
x= embedding(x)
print(x.shape)
x=layers.LSTM(64, return_sequences=True)(x) #64 is hidden units
#When you a stacking RNN Cells together, you need to return sequences=true
print(x.shape)
x= layers.LSTM(64)(x)
print(x.shape)
x= layers.Dense(64, activation='relu')(x)
print(x.shape)
outputs=layers.Dense(1, activation='sigmoid')(x)

model_2= tf.keras.Model(inputs, outputs, name="model_2_LSTM")


In [ ]:
#Create an LSTM Model
from tensorflow.keras import layers
inputs= layers.Input(shape=(1,), dtype=tf.string)
x=text_vectorizer(inputs)
x= embedding(x)
#print(x.shape)
#x=layers.LSTM(64, return_sequences=True)(x) #64 is hidden units
#When you a stacking RNN Cells together, you need to return sequences=true
#print(x.shape)
x= layers.LSTM(64)(x)
#print(x.shape)
#x= layers.Dense(64, activation='relu')(x)
#print(x.shape)
outputs=layers.Dense(1, activation='sigmoid')(x)

model_2= tf.keras.Model(inputs, outputs, name="model_2_LSTM")


In [ ]:
model_2.summary()

###The default activation function for RNNs is TanH

In [ ]:
#Complie the model
model_2.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [ ]:
history_model_2_LSTM=model_2.fit(train_sentences,
                                 train_labels,
                                 epochs=5,
                                 validation_data=(val_sentences, val_labels),
                                 callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                        experiment_name="model_2_LSTM")])

In [ ]:
model_2_pred_probs=model_2.predict(val_sentences)
model_2_pred_probs[:10]

In [ ]:
#Convert the probabs to pred
model_2_preds=tf.squeeze(tf.round(model_2_pred_probs))

In [ ]:
model_2_preds


In [ ]:
model_2_results=calculate_results(y_true=val_labels,
                                  y_pred=model_2_preds)

In [ ]:
model_2_results

In [ ]:
baseline_results


### Model 3: GRU

Similar to LSTM but has less params

In [ ]:
from tensorflow.keras import layers
inputs= tf.keras.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)

x=embedding(x)

x = tf.keras.layers.GRU(64, return_sequences=True)(x) #if you want to stack recurrent layers on top of each other, you need to use return_sequences=True
x=layers.LSTM(64, return_sequences=True)(x)
x=layers.GRU(64)(x)

x=layers.Dense(64, activation='relu')(x)
outputs=layers.Dense(1, activation='sigmoid')(x)
model_3=tf.keras.Model(inputs, outputs, name="Model_3_GRU")


In [ ]:
from tensorflow.keras import layers
inputs= tf.keras.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)

x=embedding(x)

x = tf.keras.layers.GRU(64)(x) #if you want to stack recurrent layers on top of each other, you need to use return_sequences=True
#x=layers.LSTM(64, return_sequences=True)(x)
#x=layers.GRU(64)(x)

x=layers.Dense(64, activation='relu')(x)
outputs=layers.Dense(1, activation='sigmoid')(x)
model_3=tf.keras.Model(inputs, outputs, name="Model_3_GRU")


In [ ]:
model_3.summary()

In [ ]:
model_3.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [ ]:
history_model_3_GRU=model_3.fit(train_sentences,
                                train_labels,
                                validation_data=(val_sentences, val_labels),
                                epochs=5,
                                callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                      experiment_name='model_name_GRU')])

In [141]:
model_3_pred_probs=model_3.predict(val_sentences)
model_3_pred_probs[:10]

24/24 [==============================] - 1s 8ms/step


array([[6.2693977e-03],
       [5.5768818e-01],
       [9.9999583e-01],
       [1.0364279e-01],
       [1.1872800e-05],
       [9.9975604e-01],
       [8.8811561e-02],
       [9.9999785e-01],
       [9.9999595e-01],
       [6.8013978e-01]], dtype=float32)

In [142]:
model_3_preds=tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 0., 1., 1., 1.], dtype=float32)>

In [143]:
calculate_results(y_true=val_labels,
                  y_pred=model_3_preds)

{'accuracy': 76.24671916010499,
 'precision': 0.7639905060800456,
 'recall': 0.7624671916010499,
 'f1-score': 0.7602456873168202}

In [144]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1-score': 0.7862189758049549}

### Model_4 : Bidirectional LSTM

Normal RNNs go from left to right..
Bidirectional RNNs go from left to right as well as right to left




In [159]:
from tensorflow.keras import layers
inputs=layers.Input(shape=(1,), dtype=tf.string)
x= text_vectorizer(inputs)

x= embedding(x)

x=layers.Bidirectional(layers.LSTM(64))(x)
#print(x.shape) #Doubles the value of the shape...as the cell is bidirectional
#x=layers.Bidirectional(layers.GRU(64))(x)

outputs=layers.Dense(1, activation='sigmoid')(x)

model_4=tf.keras.Model(inputs, outputs, name='model_4_bidirectional')

In [160]:
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_22 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_3 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 bidirectional_6 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense_20 (Dense)            (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,3

In [161]:
model_4.compile(loss='binary_crossentropy',
                metrics=['accuracy'],
                optimizer=tf.keras.optimizers.Adam())

In [162]:
history_4_bidirectional=model_4.fit(train_sentences,
                                    train_labels,
                                    validation_data=(val_sentences, val_labels),
                                    epochs=5,
                                    callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                           experiment_name='model_4_bidirectional')])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20230616-034334
Epoch 1/5
215/215 [==============================] - 17s 54ms/step - loss: 0.2152 - accuracy: 0.9225 - val_loss: 0.5480 - val_accuracy: 0.7835
Epoch 2/5
215/215 [==============================] - 11s 49ms/step - loss: 0.1549 - accuracy: 0.9429 - val_loss: 0.6283 - val_accuracy: 0.7822
Epoch 3/5
215/215 [==============================] - 10s 48ms/step - loss: 0.1271 - accuracy: 0.9514 - val_loss: 0.7197 - val_accuracy: 0.7743
Epoch 4/5
215/215 [==============================] - 9s 43ms/step - loss: 0.1084 - accuracy: 0.9574 - val_loss: 0.7231 - val_accuracy: 0.7769
Epoch 5/5
215/215 [==============================] - 14s 64ms/step - loss: 0.0892 - accuracy: 0.9654 - val_loss: 0.9359 - val_accuracy: 0.7677


In [163]:
model_4_pred_probs=model_4.predict(val_sentences)
model_4_pred_probs[:10]

24/24 [==============================] - 4s 8ms/step


array([[2.1251854e-01],
       [9.0274268e-01],
       [9.9991882e-01],
       [5.6247890e-02],
       [1.3182372e-04],
       [9.9746835e-01],
       [9.1958749e-01],
       [9.9996579e-01],
       [9.9990803e-01],
       [5.2167481e-01]], dtype=float32)

In [164]:
model_4_preds=tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0.,
       0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0.,
       0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1.,
       1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0

In [169]:
model_4_results=calculate_results(y_true=val_labels,
                y_pred=model_4_preds)

In [170]:
model_4_results

{'accuracy': 76.77165354330708,
 'precision': 0.7681410880728078,
 'recall': 0.7677165354330708,
 'f1-score': 0.7662770891654436}

In [171]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1-score': 0.7862189758049549}